# Fixed & random effects

## Table of contents
[Intro: Preprocessing data](#Intro:-Preprocessing-data)
   - [I.1 Load data](#I.1-Load-data)
   - [I.2 List variables](#I.2-List-variables)
   - [I.3 Choose x & y variables](#I.3-Choose-x-&-y-variables)

[Question 1:](#Question-1:)

Estimate models for infant or child mortality rates in developing countries with explanatory variables such as:GDP per capita, Total fertility rate, Health Expenditures, Immunization rates, etc. Using 5-yearly averages from the World Development Indicators data set. Select explanatory variables in the model so as to also reduce the number of missing observations in the estimation.
- [1.1 Basic regression](#1.1-Basic-regression)

[Question 2:](#Question-2:)

Explain how you account for between-country differences in the estimation methods.

- [2.1 Random effects](#2.1-Random-effects)
- [2.2 Fixed effects](#2.2-Fixed-effects)
    - [2.2.1 FE by entity](#2.2.1-FE-by-entity)
    - [2.2.2 FE by entity, time](#2.2.2-FE-by-entity,-time)

[Question 3:](#Question-3:)

Investigate if the OLS estimates differ from estimation methods that treat country specific effects as “fixed” (“dummy”) variables or “random” variables. 

- [3.1 Compare FE & RE](3.1-Compare-FE-&-RE)
- [3.2. Pooled, cluster entity, cluster entity and time](#3.2.-Pooled,-cluster-entity,-cluster-entity-and-time)


## Intro: Preprocessing data

## I.1 Load data

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_excel('WDI-subset5y_extended 2014 fall.xlsx', 
                   index_col=[0,1], 
                   parse_dates=True
                  )
dataset.head(6)

Birth rate, crude (per 1,000 people)  \
Country     Year                                               
Afghanistan 1990-01-01                             52.457400   
            1995-01-01                             52.721200   
            2000-01-01                             50.565600   
            2005-01-01                             46.737000   
            2010-01-01                             44.222667   
Albania     1990-01-01                             24.533800   

                        Births attended by skilled health staff (% of total)  \
Country     Year                                                               
Afghanistan 1990-01-01                                                NaN      
            1995-01-01                                                NaN      
            2000-01-01                                             12.400      
            2005-01-01                                             16.600      
            2010-01-01                                             24.000      
Albania     1990-01-01                                             91.175      

                        Fertility rate, total (births per woman)  \
Country     Year                                                   
Afghanistan 1990-01-01                                    7.9612   
            1995-01-01                                    8.0382   
            2000-01-01                                    7.7116   
            2005-01-01                                    6.9932   
            2010-01-01                                    6.4230   
Albania     1990-01-01                                    3.2188   

                        GDP per capita growth (annual %)  \
Country     Year                                           
Afghanistan 1990-01-01                               NaN   
            1995-01-01                               NaN   
            2000-01-01                               NaN   
            2005-01-01                          4.992695   
            2010-01-01                          6.708179   
Albania     1990-01-01                         -8.541723   

                        GDP per capita, PPP (constant 2005 international $)  \
Country     Year                                                              
Afghanistan 1990-01-01                                                NaN     
            1995-01-01                                                NaN     
            2000-01-01                                         615.042245     
            2005-01-01                                         692.240977     
            2010-01-01                                         924.171976     
Albania     1990-01-01                                        3531.372774     

                        Health expenditure per capita (current US$)  \
Country     Year                                                      
Afghanistan 1990-01-01                                          NaN   
            1995-01-01                                     0.000000   
            2000-01-01                                    14.480312   
            2005-01-01                                    28.049842   
            2010-01-01                                    34.395607   
Albania     1990-01-01                                          NaN   

                        Hospital beds (per 1,000 people)  \
Country     Year                                           
Afghanistan 1990-01-01                          0.249800   
            1995-01-01                               NaN   
            2000-01-01                          0.400000   
            2005-01-01                          0.410000   
            2010-01-01                          0.406667   
Albania     1990-01-01                          4.020920   

                        Immunization, DPT (% of children ages 12-23 months)  \
Country     Year                                                              
Afg

## I.2 List variables

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 425 entries, ('Afghanistan', Timestamp('1990-01-01 00:00:00')) to ('Zimbabwe', Timestamp('2010-01-01 00:00:00'))
Data columns (total 23 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Birth rate, crude (per 1,000 people)                          422 non-null    float64
 1   Births attended by skilled health staff (% of total)          306 non-null    float64
 2   Fertility rate, total (births per woman)                      421 non-null    float64
 3   GDP per capita growth (annual %)                              413 non-null    float64
 4   GDP per capita, PPP (constant 2005 international $)           402 non-null    float64
 5   Health expenditure per capita (current US$)                   338 non-null    float64
 6   Hospital beds (per 1,000 people)                              293 non-n

## I.3 Choose x & y variables

In [4]:
#The variableslist can be extracted from the data set by typing in: dataset.columns 

variableslist = [
       #'Birth rate, crude (per 1,000 people)',
       #'Births attended by skilled health staff (% of total)',
       #'Fertility rate, total (births per woman)',
       #'GDP per capita growth (annual %)', -Flow variable, Y is a stock variable
       #'Life expectancy at birth, total (years)', -Highly correlated with Y
       #'Primary education, pupils', -School enrolment primary is a better proxy
       #'Literacy rate, adult female (% of females ages 15 and above)', #Not enough obs
       #'Literacy rate, adult male (% of males ages 15 and above)', -Not enough obs
       #'Hospital beds (per 1,000 people)', #Explanatory variable, -Not enough obs
       #'Health expenditure per capita (current US$)', -Not enough obs
       #'School enrollment, secondary, female (% gross)', -Not enough obs
       #'School enrollment, secondary, male (% gross)', -Not enough obs
    
       #'Mortality rate, infant (per 1,000 live births)', #Y variable NOT chosen for model

#The above variables were not included in the model: 
#It includes the name of the variable and why it was not chosen    
       'Mortality rate, under-5 (per 1,000 live births)', # Y variable (Dependent variables)
       'GDP per capita, PPP (constant 2005 international $)', #Control for overall wealth       
       'Immunization, DPT (% of children ages 12-23 months)', #Treatment control
       'Immunization, measles (% of children ages 12-23 months)', #Treatment control
       'Improved sanitation facilities (% of population with access)', #General health control
       'Improved water source (% of population with access)', #General health control
       'Incidence of tuberculosis (per 100,000 people)', #Systemic disease control
       'Physicians (per 1,000 people)', #Health system control, related to wealth
       'School enrollment, primary, female (% gross)', #Education control
       'School enrollment, primary, male (% gross)', #Education control
       
]

In [5]:
dataset_cleaned = dataset[variableslist].dropna() #Drop the variables we don't wnat to include in the model
x = dataset_cleaned.drop(['Mortality rate, under-5 (per 1,000 live births)'], axis=1) #Drop the y variable to leave only explanatory variables
y = dataset_cleaned[['Mortality rate, under-5 (per 1,000 live births)']]

x = np.log(x)
x = sm.add_constant(x) #Add constant for regression
y = np.log(y)

# Question 1:
Estimate models for infant or child mortality rates in developing countries with explanatory variables such as:

GDP per capita, Total fertility rate, Health Expenditures, Immunization rates, etc. 

Using 5-yearly averages from the World Development Indicators data set. Select explanatory variables in the model so as to also reduce the number of missing observations in the estimation.d


## 1.1 Basic regression

In [6]:
#Import necessary library to run regression
from linearmodels.panel import PooledOLS #Library for panel data
import statsmodels.api as sm #Library for regression

pooled_res = PooledOLS(y,x).fit(cov_type='robust') #Robust standard errors 
pooled_res.summary #Prints summary

Dep. Variable:,"Mortality rate, under-5 (per 1,000 live births)",R-squared:,0.7520
Estimator:,PooledOLS,R-squared (Between):,0.7492
No. Observations:,300,R-squared (Within):,0.5594
Date:,"Mon, Mar 02 2020",R-squared (Overall):,0.7520
Time:,16:16:39,Log-likelihood,-129.09
Cov. Estimator:,Robust,,
,,F-statistic:,97.692
Entities:,84,P-value,0.0000
Avg Obs:,3.5714,Distribution:,"F(9,290)"
Min Obs:,0.0000,,
Max Obs:,5.0000,F-statistic (robust):,106.71


In [7]:
results_text = pooled_res.summary.as_text()

import csv
resultFile = open("table.csv",'w')
resultFile.write(results_text)
resultFile.close()

# Question 2:

## 2.1 Random effects

In [8]:
from linearmodels import RandomEffects
re_result = RandomEffects(y,x).fit(cov_type='robust') #Robust standard errors 
re_result.summary

Dep. Variable:,"Mortality rate, under-5 (per 1,000 live births)",R-squared:,0.7563
Estimator:,RandomEffects,R-squared (Between):,0.7158
No. Observations:,300,R-squared (Within):,0.6458
Date:,"Mon, Mar 02 2020",R-squared (Overall):,0.7279
Time:,16:16:39,Log-likelihood,114.38
Cov. Estimator:,Robust,,
,,F-statistic:,100.02
Entities:,84,P-value,0.0000
Avg Obs:,3.5714,Distribution:,"F(9,290)"
Min Obs:,0.0000,,
Max Obs:,5.0000,F-statistic (robust):,57.063


In [9]:
#re_result.variance_decomposition

## 2.2 Fixed effects

## 2.2.1 FE by entity

In [10]:
from linearmodels import PanelOLS
fe_result = PanelOLS(y,x, entity_effects=True).fit(cov_type='robust')
fe_result

Dep. Variable:,"Mortality rate, under-5 (per 1,000 live births)",R-squared:,0.6529
Estimator:,PanelOLS,R-squared (Between):,0.6745
No. Observations:,300,R-squared (Within):,0.6529
Date:,"Mon, Mar 02 2020",R-squared (Overall):,0.6929
Time:,16:16:39,Log-likelihood,167.49
Cov. Estimator:,Robust,,
,,F-statistic:,43.674
Entities:,84,P-value,0.0000
Avg Obs:,3.5714,Distribution:,"F(9,209)"
Min Obs:,0.0000,,
Max Obs:,5.0000,F-statistic (robust):,31.436


## 2.2.2 FE by entity, time

In [11]:
from linearmodels import PanelOLS
fe_result = PanelOLS(y,x, entity_effects=True, time_effects=True).fit(cov_type='robust')
fe_result

Dep. Variable:,"Mortality rate, under-5 (per 1,000 live births)",R-squared:,0.3324
Estimator:,PanelOLS,R-squared (Between):,0.4163
No. Observations:,300,R-squared (Within):,0.3122
Date:,"Mon, Mar 02 2020",R-squared (Overall):,0.4103
Time:,16:16:40,Log-likelihood,288.99
Cov. Estimator:,Robust,,
,,F-statistic:,11.343
Entities:,84,P-value,0.0000
Avg Obs:,3.5714,Distribution:,"F(9,205)"
Min Obs:,0.0000,,
Max Obs:,5.0000,F-statistic (robust):,7.6541


## 3.1 Compare FE & RE

In [12]:
from linearmodels.panel import compare
compare({'FE':fe_result,'RE':re_result,'Pooled regression':pooled_res}).summary

#'''Code to export table to csv'''
#compare({'FE':fe_result,'RE':re_result,'Pooled regression':pooled_res}).summary.as_csv()

,FE,RE,Pooled regression
Dep. Variable,"Mortality rate, under-5 (per 1,000 live births)","Mortality rate, under-5 (per 1,000 live births)","Mortality rate, under-5 (per 1,000 live births)"
Estimator,PanelOLS,RandomEffects,PooledOLS
No. Observations,300,300,300
Cov. Est.,Robust,Robust,Robust
R-squared,0.3324,0.7563,0.7520
R-Squared (Within),0.3122,0.6458,0.5594
R-Squared (Between),0.4163,0.7158,0.7492
R-Squared (Overall),0.4103,0.7279,0.7520
F-statistic,11.343,100.02,97.692
P-value (F-stat),0.0000,0.0000,0.0000


## 3.2. Pooled, cluster entity, cluster entity and time

In [13]:
mod = PooledOLS(y, x)
robust = mod.fit(cov_type='robust')
clust_entity = mod.fit(cov_type='clustered', cluster_entity=True)
clust_entity_time = mod.fit(cov_type='clustered', cluster_entity=True, cluster_time=True)

In [14]:
from collections import OrderedDict
res = OrderedDict()
res['Robust'] = robust
res['Entity'] = clust_entity
#res['Entity-Time'] = clust_entity_time
compare(res)

,Robust,Entity
Dep. Variable,"Mortality rate, under-5 (per 1,000 live births)","Mortality rate, under-5 (per 1,000 live births)"
Estimator,PooledOLS,PooledOLS
No. Observations,300,300
Cov. Est.,Robust,Clustered
R-squared,0.7520,0.7520
R-Squared (Within),0.5594,0.5594
R-Squared (Between),0.7492,0.7492
R-Squared (Overall),0.7520,0.7520
F-statistic,97.692,97.692
P-value (F-stat),0.0000,0.0000
